In [1]:
# import sys
# import os

# # model_study ka absolute path leke sys.path mein daal
# sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# from model_study.LMEDR import LMEDRModel

# print("Import done ✅")


In [2]:
# from model_study.LMEDR import LMEDRModel
from transformers import BartTokenizer

In [3]:
args = {
   "num_latent": 10,
   "num_latent2": 10,
}

In [4]:
from argparse import Namespace
args = Namespace(**args)

In [5]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
add_special_tokens = {'additional_special_tokens': ['<query>', '<response>', '<latent>', '<persona>']}
tokenizer.add_special_tokens(add_special_tokens)
# model = LMEDRModel.from_pretrained("facebook/bart-large", num_labels=1,
#                                            num_token=len(tokenizer),
#                                            num_latent=args.num_latent, num_latent2=args.num_latent2)
# model.resize_token_embeddings(len(tokenizer))
# model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids('<response>')
# model.config.forced_bos_token_id = None
print("Model loaded successfully ✅")

Model loaded successfully ✅


In [6]:
data_from = "_original"
args.max_history = 5
args.cand = 5

In [10]:
from build_data_PersonaChat import create_data,build_dataloader,build_infer_dataset

persona, query, response, cand = create_data(f"./data/ConvAI2/valid_self{data_from}.txt")
print("Checkpoint: Data for training created.")
train_data = build_dataloader(persona, query, response, cand, tokenizer, max_history=args.max_history, n_cand=args.cand)
print("Checkpoint: Training dataloader built.")
persona, query, response, cand = create_data(f"./data/ConvAI2/valid_self{data_from}.txt")
print("Checkpoint: Data for validation created.")
val_data = build_dataloader(persona, query, response, cand, tokenizer, max_history=args.max_history, use_all=True)
print("Checkpoint: Validation dataloader built.")
infer_data = build_infer_dataset(tokenizer, "./data/dnli/dialogue_nli_train.jsonl")
print("Checkpoint: Inference dataset built.")

./data/ConvAI2/valid_self_original.txt has 1000 dialog and 7801 query
Checkpoint: Data for training created.
Checkpoint: Training dataloader built.
./data/ConvAI2/valid_self_original.txt has 1000 dialog and 7801 query
Checkpoint: Data for validation created.
Checkpoint: Validation dataloader built.


Generate infer data: 100%|██████████| 310110/310110 [00:38<00:00, 8098.30it/s] 


Checkpoint: Inference dataset built.


In [11]:
MODEL_INPUTS = ["input_ids", "attention_mask", "lmlabels", "decoder_input_ids", "decoder_attention_mask",
                "cls_index", "clslabel", "per_input_ids", "per_attention_mask"]

INFER_INPUTS = ["encoder_input_ids", "decoder_input_ids", "attention_mask", "decoder_attention_mask",
                "lmlabels"]

trainset = []
valset = []
inferset = []

for input_name in MODEL_INPUTS:
    if input_name == "clslabel":
        tensor = train_data[input_name].view(-1)
        print("{}: {}".format(input_name, tensor.size()))
        trainset.append(tensor)

        tensor = val_data[input_name].view(-1)
        print("{}: {}".format(input_name, tensor.size()))
        valset.append(tensor)
    else:
        tensor = train_data[input_name].view(-1, args.cand, train_data[input_name].size(-1))
        print("{}: {}".format(input_name, tensor.size()))
        trainset.append(tensor)

        tensor = val_data[input_name].view(-1, 20, val_data[input_name].size(-1))
        print("{}: {}".format(input_name, tensor.size()))
        valset.append(tensor)

for input_name in INFER_INPUTS:
    tensor = infer_data[input_name].view(-1, 1, infer_data[input_name].size(-1))
    print("{}: {}".format(input_name, tensor.size()))
    inferset.append(tensor)


input_ids: torch.Size([7801, 5, 246])
input_ids: torch.Size([7801, 20, 246])
attention_mask: torch.Size([7801, 5, 246])
attention_mask: torch.Size([7801, 20, 246])
lmlabels: torch.Size([7801, 5, 30])
lmlabels: torch.Size([7801, 20, 30])
decoder_input_ids: torch.Size([7801, 5, 30])
decoder_input_ids: torch.Size([7801, 20, 30])
decoder_attention_mask: torch.Size([7801, 5, 30])
decoder_attention_mask: torch.Size([7801, 20, 30])
cls_index: torch.Size([7801, 5, 30])
cls_index: torch.Size([7801, 20, 30])
clslabel: torch.Size([7801])
clslabel: torch.Size([7801])
per_input_ids: torch.Size([7801, 5, 69])
per_input_ids: torch.Size([7801, 20, 69])
per_attention_mask: torch.Size([7801, 5, 69])
per_attention_mask: torch.Size([7801, 20, 69])
encoder_input_ids: torch.Size([100000, 1, 62])
decoder_input_ids: torch.Size([100000, 1, 42])
attention_mask: torch.Size([100000, 1, 62])
decoder_attention_mask: torch.Size([100000, 1, 42])
lmlabels: torch.Size([100000, 1, 42])


### Study the MODEL_INPUTS 

In [12]:
len(train_data['input_ids'])

39005

In [14]:
for key in train_data.keys():
    if key != 0:
       print(key,train_data[key].shape)

input_ids torch.Size([39005, 246])
attention_mask torch.Size([39005, 246])
per_input_ids torch.Size([39005, 69])
per_attention_mask torch.Size([39005, 69])
lmlabels torch.Size([39005, 30])
decoder_input_ids torch.Size([39005, 30])
decoder_attention_mask torch.Size([39005, 30])
cls_index torch.Size([39005, 30])
clslabel torch.Size([7801, 1])


In [15]:
for key in train_data.keys():
    if key != 0:
        # print(f"Exploring {key}")
        # PRinting Single Instance of Each data
        for data in train_data[key]:
            print(f"{key}:{data.shape}")
            break


input_ids:torch.Size([246])
attention_mask:torch.Size([246])
per_input_ids:torch.Size([69])
per_attention_mask:torch.Size([69])
lmlabels:torch.Size([30])
decoder_input_ids:torch.Size([30])
decoder_attention_mask:torch.Size([30])
cls_index:torch.Size([30])
clslabel:torch.Size([1])


In [16]:
for key in train_data.keys():
    if key != 0:
        # print(f"Exploring {key}")
        # PRinting Single Instance of Each data
        for data in train_data[key]:
            print(f"{key}:{data}")
            break


input_ids:tensor([50267, 50268,   939,  1166, 10328,  2799,    10,    76,     4,     2,
          939,   437,    10, 16391,  1457,    25,   127,   200,   633,     4,
            2,   939,   129,  3529, 36930,     4,     2,   939,    21,  1179,
           11,    10,   881,  4095,  6028,     4,     2, 50265, 20760,    99,
           32,   608,   452, 17487,     2,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     

In [17]:
key = "per_input_ids"
for i,data in enumerate(train_data[key]):
    print(tokenizer.decode(data))
    if i == 10:
        break

<latent> <persona>  i read twenty books a year.</s> i'm a stunt double as my second job.</s> i only eat kosher.</s> i was raised in a single parent household.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<latent> <persona>  i read twenty books a year.</s> i'm a stunt double as my second job.</s> i only eat kosher.</s> i was raised in a single parent household.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<latent> <persona>  i read twenty books a year.</s> i'm a stunt double as my second job.</s> i only eat kosher.</s> i was raised in a single parent household.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<latent> <persona>  i read twen

In [18]:
key = "input_ids"
for i,data in enumerate(train_data[key]):
    print(tokenizer.decode(data))
    if i == 10:
        break

<latent> <persona>  i read twenty books a year.</s> i'm a stunt double as my second job.</s> i only eat kosher.</s> i was raised in a single parent household.</s> <query>  hello what are doing today ?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [19]:
tokenizer.decode([939,  524,  205, 2156,  939,   95,  300,  160,  173,    8, 7428, 2156,
         939,   33,   80, 1315,  479,    2])

' i am good , i just got off work and tired , i have two jobs .</s>'

In [20]:
key = "lmlabels"
for i,data in enumerate(train_data[key]):
    print(data)
    if i == 10:
        break

tensor([ 939,  524,  205, 2156,  939,   95,  300,  160,  173,    8, 7428, 2156,
         939,   33,   80, 1315,  479,    2, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, 

In [21]:
key = "decoder_input_ids"
for i,data in enumerate(train_data[key]):
    print(tokenizer.decode(data))
    if i == 10:
        break

<response>  i am good , i just got off work and tired , i have two jobs .<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<response>  yeah , well what about you ?<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<response>  hey there , are you a mother ?<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<response>  i don't like clowns . they are scary to a kid like me<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<response>  it sure is . i'd like to see more of the city though .<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<response>  i rather read , i've read about 20 books this year .<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<response>  like me ? would you go on vacation to the beach with me<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<

In [22]:
key ="cls_index"
for i,data in enumerate(train_data[key]):
    print(data)
    if i == 10:
        break

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100,    2, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100,    2, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100, -100,    2, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100,    2, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])
tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100,    2, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, 

In [23]:
tokenizer.decode(train_data["input_ids"][0])

"<latent> <persona>  i read twenty books a year.</s> i'm a stunt double as my second job.</s> i only eat kosher.</s> i was raised in a single parent household.</s> <query>  hello what are doing today ?</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [24]:
trainset[0].shape

torch.Size([7801, 5, 246])

In [25]:
trainset[0].shape, trainset[1].shape, trainset[2].shape, trainset[3].shape, trainset[4].shape, trainset[5].shape, trainset[6].shape, trainset[7].shape

(torch.Size([7801, 5, 246]),
 torch.Size([7801, 5, 246]),
 torch.Size([7801, 5, 30]),
 torch.Size([7801, 5, 30]),
 torch.Size([7801, 5, 30]),
 torch.Size([7801, 5, 30]),
 torch.Size([7801]),
 torch.Size([7801, 5, 69]))

In [26]:
import pickle

# # Save the trainset to a pickle file
# with open('trainset.pkl', 'wb') as f_train:
#     pickle.dump(trainset, f_train)

# print("trainset saved to 'trainset.pkl'")

# Save the valset to a pickle file
with open('valset.pkl', 'wb') as f_val:
    pickle.dump(valset, f_val)

print("valset saved to 'valset.pkl'")

# Save the inferset to a pickle file
with open('inferset.pkl', 'wb') as f_infer:
    pickle.dump(inferset, f_infer)

print("inferset saved to 'inferset.pkl'")


valset saved to 'valset.pkl'
inferset saved to 'inferset.pkl'


### INFER_INPUTS dataset Study

In [ ]:
#INFER_INPUTS = ["encoder_input_ids", "decoder_input_ids", "attention_mask", "decoder_attention_mask","lmlabels"]

In [48]:
for key, data in zip(INFER_INPUTS,inferset):
    # print(f"{key}: {data.shape}")
    if key in ["encoder_input_ids", "decoder_input_ids"]:
       print(f"{key}: {data[0].squeeze(0)}")
       print(tokenizer.decode(data[0].squeeze(0)))
    elif key =="lmlabels":
       print(f"{key}: {data[0].squeeze(0)}")
    #    print(tokenizer.decode(data[0].squeeze(0)))
    else:
        print(data[0].squeeze(0))
    

encoder_input_ids: tensor([50267, 50268,   939,   524,   129,   820,    98,   939,    74,    45,
          216,   479,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])
<latent> <persona>  i am only 22 so i would not know .</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
decoder_input_ids: tensor([50266,   939,   524, 10328,    80,   107,   793,   479,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
           

In [49]:
tokenizer.decode([939,   524, 10328,    80,   107,   793,   479,     2])

' i am twenty two years old .</s>'

## Temp area

In [ ]:
## This Lines Taking too much memory with train dataset

# persona, query, response, cand = create_data(f"../data/ConvAI2/train_self{data_from}.txt")
# print("Checkpoint: Data for training created.")
# train_data = build_dataloader(persona, query, response, cand, tokenizer, max_history=args.max_history, n_cand=args.cand)
# print("Checkpoint: Training dataloader built.")

### Temp Area

In [10]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.distributed import DistributedSampler
import logging

args.distributed = False  # Set to True if using distributed training
args.train_batch_size = 2
args.valid_batch_size = 2
args.infer_batch_size = 32

# Initialize logger
logger = logging.getLogger(__name__)

# Assuming trainset, valset, and inferset are already defined and are tuples of tensors
train_dataset = TensorDataset(*trainset)
val_dataset = TensorDataset(*valset)
infer_dataset = TensorDataset(*inferset)

logger.info("Prepare dataloader.")

# Set up DistributedSampler for distributed training, if applicable
train_sampler = DistributedSampler(train_dataset) if args.distributed else None
infer_sampler = DistributedSampler(infer_dataset) if args.distributed else None
val_sampler = DistributedSampler(val_dataset) if args.distributed else None

# Create DataLoader for training, inference, and validation
train_loader = DataLoader(
    train_dataset, 
    sampler=train_sampler, 
    batch_size=args.train_batch_size,
    shuffle=(not args.distributed), 
    num_workers=0
)

infer_loader = DataLoader(
    infer_dataset, 
    sampler=infer_sampler, 
    batch_size=args.infer_batch_size,
    shuffle=(not args.distributed), 
    num_workers=0
)

val_loader = DataLoader(
    val_dataset, 
    sampler=val_sampler, 
    batch_size=args.valid_batch_size, 
    shuffle=False
)


In [41]:
for data in train_loader:
    print("Train Loader Batch Shapes:")
    for tensor in data:
        print(tensor.shape)
    break

for data in val_loader:
    print("\nValidation Loader Batch Shapes:")
    for tensor in data:
        print(tensor.shape)
    break

for data in infer_loader:
    print("\nInfer Loader Batch Shapes:")
    for tensor in data:
        print(tensor.shape)
    break


Train Loader Batch Shapes:
torch.Size([2, 5, 246])
torch.Size([2, 5, 246])
torch.Size([2, 5, 30])
torch.Size([2, 5, 30])
torch.Size([2, 5, 30])
torch.Size([2, 5, 30])
torch.Size([2])
torch.Size([2, 5, 69])
torch.Size([2, 5, 69])

Validation Loader Batch Shapes:
torch.Size([2, 20, 246])
torch.Size([2, 20, 246])
torch.Size([2, 20, 30])
torch.Size([2, 20, 30])
torch.Size([2, 20, 30])
torch.Size([2, 20, 30])
torch.Size([2])
torch.Size([2, 20, 69])
torch.Size([2, 20, 69])

Infer Loader Batch Shapes:
torch.Size([32, 1, 62])
torch.Size([32, 1, 42])
torch.Size([32, 1, 62])
torch.Size([32, 1, 42])
torch.Size([32, 1, 42])


In [45]:
from transformers import AdamW

args.lr = 5e-5

memory1_params = list(map(id, model.memory1))
base_params = filter(lambda p: id(p) not in memory1_params, model.parameters())

optimizer_infer = AdamW(model.parameters(), lr=args.lr, correct_bias=True)
optimizer_bart = AdamW(base_params, lr=args.lr, correct_bias=True)


In [48]:
from ignite.engine import Engine, Events
args.max_grad_norm = 1.0
args.device = "cuda" if torch.cuda.is_available() else "cpu"
def infer_update(engine, batch):
        model.train()
        batch = tuple(input_tensor.to(args.device) for input_tensor in batch)
        infer_input_ids, infer_decoder_input_ids, infer_attention_mask, \
        infer_decoder_attention_mask, infer_lmlabels = batch
        outputs = model(infer_input_ids=infer_input_ids,
                        infer_decoder_input_ids=infer_decoder_input_ids,
                        infer_attention_mask=infer_attention_mask,
                        infer_lmlabels=infer_lmlabels,
                        infer_decoder_attention_mask=infer_decoder_attention_mask
                        )
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
        optimizer_infer.step()
        optimizer_infer.zero_grad()
        return {'loss': loss.item()}

infer_trainer = Engine(infer_update)

In [49]:
args.epochs = 1

@infer_trainer.on(Events.STARTED)
def log_training_start(engine):
    print("Training started...")

@infer_trainer.on(Events.COMPLETED)
def log_training_end(engine):
    print("Training completed.")

@infer_trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    print(f"Epoch {engine.state.epoch} - Loss: {engine.state.output['loss']:.4f}")

@infer_trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    if engine.state.iteration % 100 == 0:
        print(f"Iteration {engine.state.iteration} - Loss: {engine.state.output['loss']:.4f}")

infer_trainer.run(infer_loader, max_epochs=args.epochs)


Training started...


: 